In [1]:
import os
import numpy as np
import random

In [2]:
# classes_name =  ['Car', 'Van', 'Truck', 'Pedestrian', 'Person_sitting', 'Cyclist', 'Tram', 'Misc']
classes_name =  ['Car', 'Van', 'Truct', 'Pedestrian', 'Person_sitting', 'Cyclist']

classes_num = {}
for idx, class_name in enumerate(classes_name):
    classes_num[class_name] = idx

truncate_th = 0.5
occlude_th = 2

In [3]:
IMAGE_PATH = 'kitti/training/image_2'
DATA_PATH = 'kitti/training/label_2'
test_ratio = 0.1
train_file_name = "kitti_train.txt"
test_file_name = "kitti_test.txt"

In [4]:
def convert_to_string(image_path, labels):
    """convert image_path, lables to string 
    Returns:
    string 
    """
    out_string = ''
    out_string += image_path
    for label in labels:
        for i in label:
            out_string += ' ' + str(i)
    out_string += '\n'
    return out_string

In [9]:
effect_img = 0
effect_obj = 0
max_obj = 0
records = []
for (dirpath, dirnames, filenames) in os.walk(DATA_PATH):
    for file_idx, filename in enumerate(filenames):
        image_filename = filename.replace('txt', 'png')
        image_path = os.path.join(IMAGE_PATH, image_filename)
        
        filepath= os.path.join(dirpath, filename)
        
        with open(filepath) as label_file:
            objs = label_file.readlines()
        
        formatted_objs = []
        for obj in objs:
            [klass, truncated, occluded, _, xmin, ymin, xmax, ymax, _, _, _, _, _, _, _] = obj.split(" ")
            if klass not in classes_name:
                continue
            if float(truncated) > truncate_th:
                continue
            if int(occluded) >= occlude_th:
                continue
            klass_idx = classes_num[klass]
            formatted_objs.append([xmin, ymin, xmax, ymax, str(klass_idx)])
            effect_obj += 1
        
        if len(formatted_objs) == 0:
            continue
        
        if len(formatted_objs) > max_obj:
            max_obj = len(formatted_objs)
        
        effect_img += 1
        
        record = convert_to_string(image_path, formatted_objs)
        
        records.append(record)
#         train_out_file.write(record)

print("Effective images: " + str(effect_img))
print("Effective objects: " + str(effect_obj))
print("Max objects in : " + str(max_obj))


Effective images: 7471
Effective objects: 26099
Max objects in : 19


In [10]:
random.shuffle(records)
print(len(records))

7471


In [11]:
# split into training set and test set
total_num = len(records)
test_num = int(test_ratio * total_num)
train_num = total_num - test_num
train_records = records[0:train_num]
test_records = records[train_num:]

In [12]:
# save to text file
train_out_file = open(train_file_name, "w")
for record in train_records:
    train_out_file.write(record)
train_out_file.close()
test_out_file = open(test_file_name, "w")
for record in test_records:
    test_out_file.write(record)
test_out_file.close()